# Wildfire Detection



## 1) Data loading

In [4]:
import json
import pandas as pd


def load_data(filename, name=None):
        with open('data/' + filename, 'r') as f:
            data = json.load(f)
        
        dfs = {
            "info": pd.DataFrame(data["info"], index=[0]),
            "licenses": pd.DataFrame(data["licenses"], index=[0]),
            "categories": pd.DataFrame(data["categories"]),
            "images": pd.DataFrame(data["images"]),
            "annotations": pd.DataFrame(data["annotations"])
        }

       
        if name:
            if name not in dfs:
                raise ValueError(f"Nom invalide : {name}. Choisis parmi {list(dfs.keys())}")
            return dfs[name]

        return dfs

df_info = load_data("_annotations.coco.json","info")
df_licenses= load_data("_annotations.coco.json","licenses")
df_categories= load_data("_annotations.coco.json","categories")
df_images = load_data("_annotations.coco.json","images")
df_annotations= load_data("_annotations.coco.json","annotations")

## 2) Data Exploration

La fonction ext_file permet de retourner le nombre de fichiers présents dans un dossier ainsi que leur extension de fichier distinctes.

In [5]:
import os

def ext_file(dir):
    path = str(os.getcwd()) + dir

    files = os.listdir(path)
    files = [f for f in files if os.path.isfile(os.path.join(path, f))]
    list_ext=[]
    for f in files:
        root, ext = os.path.splitext(f)
        list_ext.append(ext)
    print(list_ext.count('.jpg'))
    print(list_ext.count('.json'))

ext_file("/data")

500
1


In [48]:
def compare_files(file1, file2):
    path = str(os.getcwd()) + file1

    files = os.listdir(path)
    files = [f for f in files if os.path.isfile(os.path.join(path, f))]
    list_files=[]
    for f in files:
        list_files.append(f)
    list_files.remove("_annotations.coco.json") # Il faut remove le json de la liste
    df_files = pd.DataFrame(list_files, columns=["file_name"])
    set1 = set(df_files['file_name'])
    set2 = set(file2)

    return len(set1), len(set2),set1 == set2

compare_files("/data/", df_images["file_name"])

(500, 500, True)

In [6]:
df_images.info()
df_images.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             500 non-null    int64 
 1   license        500 non-null    int64 
 2   file_name      500 non-null    object
 3   height         500 non-null    int64 
 4   width          500 non-null    int64 
 5   date_captured  500 non-null    object
 6   extra          500 non-null    object
dtypes: int64(4), object(3)
memory usage: 27.5+ KB


,id,license,height,width
count,500.000000,500.0,500.0,500.0
mean,249.500000,1.0,860.0,1200.0
std,144.481833,0.0,0.0,0.0
min,0.000000,1.0,860.0,1200.0
25%,124.750000,1.0,860.0,1200.0
50%,249.500000,1.0,860.0,1200.0
75%,374.250000,1.0,860.0,1200.0
max,499.000000,1.0,860.0,1200.0


In [7]:
print(df_images["id"].duplicated().sum(), "Doublons d'images")
print(df_images["date_captured"].duplicated().sum(), "Doublons de date")

0 Doublons d'images
499 Doublons de date


In [8]:
df_annotations.info()
df_annotations.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            959 non-null    int64  
 1   image_id      959 non-null    int64  
 2   category_id   959 non-null    int64  
 3   bbox          959 non-null    object 
 4   area          959 non-null    float64
 5   segmentation  959 non-null    object 
 6   iscrowd       959 non-null    int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 52.6+ KB


,id,image_id,category_id,area,iscrowd
count,959.000000,959.000000,959.0,9.590000e+02,959.0
mean,479.000000,248.694473,1.0,1.016455e+05,0.0
std,276.983754,144.208724,0.0,1.656238e+05,0.0
min,0.000000,1.000000,1.0,2.070000e-01,0.0
25%,239.500000,123.000000,1.0,1.044102e+04,0.0
50%,479.000000,247.000000,1.0,3.462144e+04,0.0
75%,718.500000,377.500000,1.0,1.135703e+05,0.0
max,958.000000,499.000000,1.0,1.032000e+06,0.0


In [9]:
print(df_annotations["image_id"].duplicated().sum(), "Doublons dans images_id de annotations")
print(df_annotations["id"].duplicated().sum(), "Doublons dans id de annotations")
print(df_annotations["area"].duplicated().sum(), "Doublons de area de annotations")

466 Doublons dans images_id de annotations
0 Doublons dans id de annotations
3 Doublons de area de annotations


On voit qu'il y a dans le JSON:\
500 images au total, pour 959 annotations

Il y a 2 catégories : 1 fire avec une supercategory wildfire, et une wildfire avec none supercategory.\
Toutes les images dans annotations sont en catégorie 1, "fire" avec supercategory "wildfire"

Il y a aussi une variable "iscrowd". Toutes les images sont à 0 en iscrowd.\
Toutes les images sont au même format : 860x1200

Toutes les images ont la même date, et elles sont toutes en JPEG\
Il y a aussi 3 doublons d'area

Il faut récupérer le nom des images dans le JSON et le nom des images dans le dossier data et comparer

In [10]:
def files_extension(folder):

    files = os.listdir(str(os.getcwd()) + folder)
    files = [f for f in files if os.path.isfile(os.path.join(str(os.getcwd()) + folder, f))]
    list_ext = [os.path.splitext(f)[1] for f in files]

    return set(list_ext)

files_extension("/data/")

{'.jpg', '.json'}

In [11]:
import os

def compare_files(file1, file2):
    path = str(os.getcwd()) + file1

    files = os.listdir(path)
    files = [f for f in files if os.path.isfile(os.path.join(path, f))]
    list_root=[]
    for f in files:
        root, ext = os.path.splitext(f)
        list_root.append(root)
    list_root.remove("_annotations.coco") # Il faut remove le json de la liste
    df_root = pd.DataFrame(list_root, columns=["file_name"])
    set1 = set(df_root['file_name'])
    set2 = set(file2)

    return len(set1), len(set2),set1 == set2

In [12]:
compare_files("/data", df_images["file_name"])

(500, 500, False)

In [13]:
print(df_images["file_name"].duplicated().sum(), "Doublons de noms de fichier")

0 Doublons de noms de fichier


In [14]:
df_annot_image_id = df_annotations["image_id"].astype(int)
display(df_annot_image_id.value_counts())

image_id
466    10
439     9
425     7
452     7
48      7
       ..
24      1
27      1
28      1
30      1
31      1
Name: count, Length: 493, dtype: int64

On remarque qu'une image peut avoir plusieurs annotations. On veut maintenant obtenir le nombre d'image différentes dans annotations ainsi que celles manquantes dans annotations.

In [15]:
df_annot_image_id.nunique()

493

Il y a 493 images différentes dans annotations. Il y a donc 7 images manquantes qui n'ont pas d'annotations. 

In [16]:
list_annot_id = df_annot_image_id.tolist()
list_image_id = df_images["id"].tolist()


list_diff = []
for z in list_image_id:
    if z not in list_annot_id:
        list_diff.append(z)
print(list_diff)

[0, 96, 118, 122, 209, 290, 451]


0, 96, 118, 122, 209, 290 et 451 sont les images non annotées. On n'a donc pas de category ID pour ces images.

In [17]:
df_images_n_annot_u = df_images.loc[df_images["id"].isin(list_diff)]
display(df_images_n_annot_u)

,id,license,file_name,height,width,date_captured,extra
0,0,1,cl6e1qges001kgk555z158f33_2_FALSE_COLOR_jpg.rf...,860,1200,2025-09-14T12:06:19+00:00,{'name': 'cl6e1qges001kgk555z158f33_2_FALSE_CO...
96,96,1,cl6kf5xzo000gc4552qc8hhcc_2_FALSE_COLOR_jpg.rf...,860,1200,2025-09-14T12:06:19+00:00,{'name': 'cl6kf5xzo000gc4552qc8hhcc_2_FALSE_CO...
118,118,1,cl6e2kygp002egk55asrs2brz_1_TRUE_COLOR_jpg.rf....,860,1200,2025-09-14T12:06:19+00:00,{'name': 'cl6e2kygp002egk55asrs2brz_1_TRUE_COL...
122,122,1,cl6e3enfn003sgk554uim9wo6_1_TRUE_COLOR_jpg.rf....,860,1200,2025-09-14T12:06:19+00:00,{'name': 'cl6e3enfn003sgk554uim9wo6_1_TRUE_COL...
209,209,1,cl6kfx47x001tc45578ts0yz6_2_FALSE_COLOR_jpg.rf...,860,1200,2025-09-14T12:06:19+00:00,{'name': 'cl6kfx47x001tc45578ts0yz6_2_FALSE_CO...
290,290,1,cl6b5x63r005il4551chxdt93_2_FALSE_COLOR_jpg.rf...,860,1200,2025-09-14T12:06:19+00:00,{'name': 'cl6b5x63r005il4551chxdt93_2_FALSE_CO...
451,451,1,cl6b5myi60048l45530d5anq4_1_TRUE_COLOR_jpg.rf....,860,1200,2025-09-14T12:06:19+00:00,{'name': 'cl6b5myi60048l45530d5anq4_1_TRUE_COL...


## Résumé : 

data folder : 
Il y a 500 image files dans /data

json/annotations : 
Il y a 959 ids différents, liés à seulement 493 unique nom de fichiers (images_id)\
Les anomalies sont : 0, 96, 118, 122, 209, 290, 451

Seulement 1 fichier n'a ni doublons, ni annotations : 0

bbox = [x, y, w, h]\
area = w*h


In [18]:
print(df_annotations["bbox"].duplicated().sum())
print(df_annotations[df_annotations["bbox"].duplicated()])
print(df_annotations["area"].duplicated().sum())
print(df_annotations[df_annotations["area"].duplicated()])

2
      id  image_id  category_id                    bbox       area  \
200  200       104            1       [0, 0, 1200, 860]  1032000.0   
581  581       300            1  [0, 111, 1200, 749.16]   898992.0   

    segmentation  iscrowd  
200           []        0  
581           []        0  
3
      id  image_id  category_id                    bbox       area  \
200  200       104            1       [0, 0, 1200, 860]  1032000.0   
570  570       291            1   [367, 0, 564.24, 860]   485246.4   
581  581       300            1  [0, 111, 1200, 749.16]   898992.0   

    segmentation  iscrowd  
200           []        0  
570           []        0  
581           []        0  


Il y a 2 doublons de bbox : images 104 et 300.\
Il y a 3 doublons de area : images 104, 291 et 300

La fonction bbox_test test si les origines (x,y) sont supérieurs ou égales à 0, et si les largeur et hauteur (w, h) sont supérieures à 0.

In [19]:
def minsize_test():
    test_x, test_y, test_w, test_h = zip(*[(i[0] >= 0, i[1] >= 0, i[2] > 0, i[3] > 0) for i in df_annotations["bbox"].apply(lambda x: x[0:4])])
    return all(test_x), all(test_y), all(test_w), all(test_h)

minsize_test()

(True, True, True, True)

In [ ]:
def maxsize_test():
    test_x, test_y, test_w, test_h = zip(*[(i[0] <= 1200, i[1] <= 860, i[2] <= 1200, i[3] <= 860) for i in df_annotations["bbox"].apply(lambda x: x[0:4])])
    return all(test_x), all(test_y), all(test_w), all(test_h)

maxsize_test()

(True, True, True, True)

Les images sont bien toutes dans la range.\
On va donc tester les images qui n'ont pas de bbox/area et celles qui ont tout

In [53]:
def invalid_values():

    """Vérifies les champs des valeurs du df_annotations pour les colonnes bbox et area.

    Returns:
        _type_: Booleans et prints
    """
    testmax_x, testmax_y, testmax_w, testmax_h = zip(*[(i[0] <= 1200, i[1] <= 860, i[2] <= 1200, i[3] <= 860) for i in df_annotations["bbox"].apply(lambda x: x[0:4])])
    
    testmin_x, testmin_y, testmin_w, testmin_h = zip(*[(i[0] >= 0, i[1] >= 0, i[2] > 0, i[3] > 0) for i in df_annotations["bbox"].apply(lambda x: x[0:4])])

    testmax_area = df_annotations.loc[(df_annotations["area"] >= 1032000)]

    testmin_area = df_annotations.loc[(df_annotations["area"] <= 0)]

    return print(f"{all(testmax_x)} max X, {all(testmax_y)} max Y, {all(testmax_w)} max W, {all(testmax_h)} max H, {all(testmin_x)} min X, {all(testmin_y)} min Y, {all(testmin_w)} min W, {all(testmin_h)} min H, {all(testmax_area)} max area, {all(testmin_area)} min area")

invalid_values()

True max X, True max Y, True max W, True max H, True min X, True min Y, True min W, True min H, True max area, True min area


In [21]:
df_annotations.loc[(df_annotations["bbox"].apply(lambda x: x[3:4][0] ==  860)) & (df_annotations["bbox"].apply(lambda x: x[2:3][0] ==  1200))]

,id,image_id,category_id,bbox,area,segmentation,iscrowd
60,60,30,1,"[0, 0, 1200, 860]",1032000.0,[],0
200,200,104,1,"[0, 0, 1200, 860]",1032000.0,[],0


In [22]:
df_annotations.loc[(df_annotations["bbox"].apply(lambda x: x[3:4][0] ==  0)) & (df_annotations["bbox"].apply(lambda x: x[2:3][0] ==  0))]

,id,image_id,category_id,bbox,area,segmentation,iscrowd


In [23]:
df_annotations.loc[(df_annotations["area"] >= 1032000)]

,id,image_id,category_id,bbox,area,segmentation,iscrowd
60,60,30,1,"[0, 0, 1200, 860]",1032000.0,[],0
200,200,104,1,"[0, 0, 1200, 860]",1032000.0,[],0


In [24]:
df_annotations.loc[(df_annotations["area"] <= 0)]

,id,image_id,category_id,bbox,area,segmentation,iscrowd


Il n'y a pas d'aberrations dans les valeurs des bbox ou de l'area. On va maintenant ouvrir les images pour voir s'il y a des aberrations et s'intéresser aux valeurs de bbox et area, ainsi que regarder nos doublons.

0, 96, 118, 122, 209, 290, 451 : sans annotations\
60, 200 : full size area

In [25]:
files_name = df_images["file_name"].loc[df_images["id"].isin([0, 60, 96, 118, 122, 200, 209, 290, 451])].tolist()
files_name

['cl6e1qges001kgk555z158f33_2_FALSE_COLOR_jpg.rf.479904c9e54c6ba121689341598bf3ed.jpg',
 'cl6b78q1y009sl455ekwc1jfp_1_TRUE_COLOR_jpg.rf.92ebe9452b2cdcf4ae4701102a55bd98.jpg',
 'cl6kf5xzo000gc4552qc8hhcc_2_FALSE_COLOR_jpg.rf.8fbfb16c6d86076d860904397218acd5.jpg',
 'cl6e2kygp002egk55asrs2brz_1_TRUE_COLOR_jpg.rf.2a198a37785242fb6db3ef21c0ffdcad.jpg',
 'cl6e3enfn003sgk554uim9wo6_1_TRUE_COLOR_jpg.rf.beaf169c01f4596cb24c1c4628ef21f1.jpg',
 'cl6kgeqar002lc455akv50au4_2_FALSE_COLOR_jpg.rf.cc30b23aec838a549eeb61aeb281a5f9.jpg',
 'cl6kfx47x001tc45578ts0yz6_2_FALSE_COLOR_jpg.rf.5a9501946306fabdad807a6acb61ccbe.jpg',
 'cl6b5x63r005il4551chxdt93_2_FALSE_COLOR_jpg.rf.8c4fb06b6812031edb1ad781ec14d2b9.jpg',
 'cl6b5myi60048l45530d5anq4_1_TRUE_COLOR_jpg.rf.1b9e229b9d66a61c1d9a410fd8ad6f23.jpg']

## En conclusion : 

D'après l'analyse de données, on décide de drop les 7 images sans annotations.

- Quel(s) type(s) de modèle(s) permet(tent) de répondre à la problématique du projet ?\
Modèle de détection d'objets
- Quelles sont les spécificités d’un modèle pré-entraîné ?\
Il est plus rapide à entraîner, moins gourmand en ressources
- Quelles sont les principales différences architecturales entre un CPU et un GPU ? Comment est équipé votre ordinateur ?\
Un CPU est beaucoup plus petit et possède beaucoup moins de coeurs. Un GPU est en général plus gros, plus rapide.\
Nos ordinateurs sont équipés d'un CPU 4 coeurs i7 10ème gen à 1.3 GHz, et de GE Force GTX 1650
- Comment la vitesse d'entraînement de YOLO varie-t-elle entre CPU et GPU ?\
Beaucoup. Les GPU sont plus beaucoup plus rapides.
- Quels modèles privilégier pour notre problématique en prenant en compte la taille du dataset ? Les contraintes de ressources d'entraînement ? \
Il faut privilégier les modèles de détection d'objets vis à vis de notre problématique.
- Quelles solutions existent pour adapter la taille de notre dataset si c’est nécessaire au modèle ?\
On peut créer des fausses données à partir de nos données existentes ou faire de la cross evaluation pour varier l'entraînement.


- Recall/rappel : Score qui identifie tout les True Positives.
- Precision : Score qui identifie tout les Positives.
- IoU : C'est la précision pour notre détection d'objet. Elle mesure à quelle point la détection est précise.
- mAP : C'est la moyenne de notre précision sur tout nos objets.

## 3) Modèle

Data preparation pour le modèle YOLO de ultralytics

In [ ]:
from pathlib import Path

path = Path('.').absolute()
subdir = ["test", "train", "val"]

def test_train_val_dirs():
    for x in subdir:
        Path(x+"/images").mkdir(parents=True)
        Path(x+"/labels").mkdir(parents=True)

test_train_val_dirs(path)


In [ ]:
def list_img():
    
    list_img = list(p.glob('**/*.jpg'))
    print(list_img)

list_img()

[PosixPath('data/cl6kf110z0004c455dymnacxa_1_TRUE_COLOR_jpg.rf.84a69e729c116c44303cdbf3fc321a71.jpg'), PosixPath('data/cl6opbnv8006xao55hbpe2j2u_1_TRUE_COLOR_jpg.rf.a322c310c87b9b2b0dd656c0db4e6c91.jpg'), PosixPath('data/cl6ojbg9t004oao552glqe340_1_TRUE_COLOR_jpg.rf.537df0a4cd6f7c528f9293ccb4d08783.jpg'), PosixPath('data/cl6e0kgem0014gk55cl4692g7_1_TRUE_COLOR_jpg.rf.d47055293ccf936f71eb03a282b643b4.jpg'), PosixPath('data/cl6ooxizh0068ao55h1l3hrim_1_TRUE_COLOR_jpg.rf.73178115b18484ae56027970999c67a5.jpg'), PosixPath('data/cl6cgs3qs00bil4551v5n3v2h_2_FALSE_COLOR_jpg.rf.dfd038db2b2cc23577079525dec70528.jpg'), PosixPath('data/cl6lsxiqd001v8w55fgtjavib_2_FALSE_COLOR_jpg.rf.ed583532db782f8ccd0047e2abe0bee1.jpg'), PosixPath('data/cl6ojv5dx0056ao557rpa0ljr_2_FALSE_COLOR_jpg.rf.e62e4cd6da325f259c3e6b98c8f16dd1.jpg'), PosixPath('data/cl6kfyj4a001wc4558emoaaeq_4_FALSE_COLOR__URBAN_jpg.rf.f4bb8a00db7566082ef5532e62ffc462.jpg'), PosixPath('data/cl6oe0y5h002rao55ghk1g0ha_4_FALSE_COLOR__URBAN_jpg.rf.

In [ ]:
from ultralytics.data.converter import convert_coco
from prepare_data.data_loader import load_dataframes, load_json
from sklearn.model_selection import train_test_split
from pathlib import Path

path = Path('.')
target_dir = "dataset"
subdir = ["test", "train", "val"]

def create_targets():

    convert_coco(
        labels_dir = "data_filtered",
        save_dir= target_dir,
        use_segments = False,
        use_keypoints = False,
        cls91to80 = True,
        lvis = False,
    )

    for x in subdir:
        Path("dataset/"+x+"/images").mkdir(parents=True)
        Path("dataset/"+x+"/labels").mkdir(parents=True)

create_targets()

Annotations /home/achar/test_bash/wildfire-detection/data_filtered/_annotations_filtered.coco.json: 100% ━━━━━━━━━━━━ 493/493 6.3Kit/s 0.1s
COCO data converted successfully.
Results saved to /home/achar/test_bash/wildfire-detection/dataset
493 [PosixPath('data_filtered/cl6kf110z0004c455dymnacxa_1_TRUE_COLOR_jpg.rf.84a69e729c116c44303cdbf3fc321a71.jpg'), PosixPath('data_filtered/cl6opbnv8006xao55hbpe2j2u_1_TRUE_COLOR_jpg.rf.a322c310c87b9b2b0dd656c0db4e6c91.jpg'), PosixPath('data_filtered/cl6ojbg9t004oao552glqe340_1_TRUE_COLOR_jpg.rf.537df0a4cd6f7c528f9293ccb4d08783.jpg'), PosixPath('data_filtered/cl6e0kgem0014gk55cl4692g7_1_TRUE_COLOR_jpg.rf.d47055293ccf936f71eb03a282b643b4.jpg'), PosixPath('data_filtered/cl6ooxizh0068ao55h1l3hrim_1_TRUE_COLOR_jpg.rf.73178115b18484ae56027970999c67a5.jpg'), PosixPath('data_filtered/cl6cgs3qs00bil4551v5n3v2h_2_FALSE_COLOR_jpg.rf.dfd038db2b2cc23577079525dec70528.jpg'), PosixPath('data_filtered/cl6lsxiqd001v8w55fgtjavib_2_FALSE_COLOR_jpg.rf.ed583532db782f8c

([PosixPath('data_filtered/cl6cttw6o00ial4555urs554w_2_FALSE_COLOR_jpg.rf.d92cb74564bf4e1556a74fc7b377e3f2.jpg'),
  PosixPath('data_filtered/cl6m10x9x00348w55h67q3hes_2_FALSE_COLOR_jpg.rf.746f55b2cb2a94835ebbb485b3715504.jpg'),
  PosixPath('data_filtered/cl6kj4n7o000o8w55eq83czxi_1_TRUE_COLOR_jpg.rf.ed5093deefd129171df94fb32cf9488d.jpg'),
  PosixPath('data_filtered/cl6b79f3e009vl455hmyk8e9e_1_TRUE_COLOR_jpg.rf.3011a8be79452c723e8e816eae6c3377.jpg'),
  PosixPath('data_filtered/cl6e0kgem0014gk55cl4692g7_1_TRUE_COLOR_jpg.rf.d47055293ccf936f71eb03a282b643b4.jpg'),
  PosixPath('data_filtered/cl6kfnwqd0015c455cssv4ha4_1_TRUE_COLOR_jpg.rf.dd4a1a9cc9e885b3011de03732c8f204.jpg'),
  PosixPath('data_filtered/cl6cg8u0w00ajl455a3ho7vov_4_FALSE_COLOR__URBAN_jpg.rf.727d492c1d7851360cea6ef50154ae46.jpg'),
  PosixPath('data_filtered/cl6kfbq11000yc45577e7e4bg_1_TRUE_COLOR_jpg.rf.c8c9d7c11989ad75fba7ee813467b034.jpg'),
  PosixPath('data_filtered/cl6orry9200a0ao559gb82ms0_1_TRUE_COLOR_jpg.rf.cc9619d3a597c

In [ ]:
from ultralytics.data.converter import convert_coco
from prepare_data.data_loader import load_dataframes, load_json
from sklearn.model_selection import train_test_split
from pathlib import Path
import shutil

path = Path('.')
target_dir = "dataset"
origin_dir = "data_filtered"
subdir = ["test", "train", "val"]

def test_split(origin, target):

    imgs = list(path.glob(origin+'/*.jpg'))
    txts = list(path.glob(target+'/**/*.txt'))

    img_train, img_test, txt_train, txt_test = train_test_split(imgs, txts, test_size=0.3, random_state=42)
    img_val, img_test, txt_val, txt_test = train_test_split(img_test, txt_test, test_size=0.3, random_state=42)
    
    for x in img_train:
        shutil.copy(x, target_dir+"/train/images")
    for x in img_test:
        shutil.copy(x, target_dir+"/test/images")
    for x in txt_train:
        shutil.copy(x, target_dir+"/train/labels")
    for x in txt_test:
        shutil.copy(x, target_dir+"/test/labels")
    for x in img_val:
        shutil.copy(x, target_dir+"/val/images")
    for x in txt_val:
        shutil.copy(x, target_dir+"/val/labels")

test_split(origin_dir, target_dir)